In [ ]:
import pandas as pd
import csv
from collections import Counter

In this notebook we combine different models to obtain the best performance.

To do so, we first load each model's predictions.

In [ ]:
fasttext = pd.read_csv('predictions/predictions_fasttext_7946.csv')

mdeberta = pd.read_csv('predictions/predictions_mdebertav3base_86293.csv')

rembert   = pd.read_csv('predictions/predictions_rembert_87445.csv')

rembert2 = pd.read_csv('predictions/predictions_rembert_8683.csv')

xlmroberta = pd.read_csv('predictions/predictions_xlmlast2_87101.csv')

bertmulti  = pd.read_csv('predictions/predictions_bertmulti_8656.csv')

bert = pd.read_csv('predictions/predictions_bert_8267.csv')

# xlmroberta = pd.read_csv('predictions/predictions_xlmroberta_8647.csv')
# tfidf = pd.read_csv('predictions/predictions_tf_lg_7432.csv')
# tfidf2 = pd.read_csv('predictions/predictions_tf_sgc_73898.csv')
# mt5 = pd.read_csv('predictions/predictions_mt5_7931.csv')

In [ ]:
df_ensemble = pd.DataFrame()
df_ensemble['ID'] = rembert['ID']
df_ensemble['mdeberta'] = mdeberta['Label']
df_ensemble['rembert'] = rembert['Label']
df_ensemble['xlmroberta'] = xlmroberta['Label']
df_ensemble['bertmulti'] = bertmulti['Label']
df_ensemble['bert'] = bert['Label']
df_ensemble['fasttext'] = fasttext['Label']
df_ensemble['rembert2'] = rembert2['Label']
# df_ensemble['mt5'] = mt5['Label']
# df_ensemble['tfidf'] = tfidf['Label']

On implémente la fonction de vote de la méthode d'ensemble learning.

In [ ]:
def vote(row):

    votes = [row['mdeberta'], row['rembert'], row['xlmroberta'], row['bertmulti'], row['bert'], row['rembert2'], row['fasttext']]
    compteur = Counter(votes)
    max_votes = max(compteur.values())
    # find labels with max number of votes
    labels_max = [label for label, count in compteur.items() if count == max_votes]
    
    # If there is a majoritary label, we return it
    if len(labels_max) == 1:
        return labels_max[0]
    else:
        # In case of equality we return the label of RemBERT
        if row['rembert'] in labels_max:
            return row['rembert']
        elif row['xlmroberta'] in labels_max:
            return row['xlmroberta']
        elif row['rembert2'] in labels_max:
            return row['rembert2']
        elif row['bertmulti'] in labels_max:
            return row['bertmulti']
        elif row['mdeberta'] in labels_max:
            return row['mdeberta']
        else:
            return labels_max[0]

# The priorisation order is the following: RemBERT then xlmroberta then rembert2 then bertmulti then mdeberta

In [ ]:
df_ensemble['Final_Label'] = df_ensemble.apply(vote, axis=1)

submission = df_ensemble[['ID', 'Final_Label']].rename(columns={'Final_Label': 'Label'})

submission.to_csv('predictions/final_submission.csv', index=False)